In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
import numpy as np
import pandas as pd

import io

import tensorflow as tf

In [3]:
tf.logging.set_verbosity(tf.logging.ERROR)
tf.set_random_seed(1)

### Load data

In [4]:
# download data
train_url = 'https://download.mlcc.google.com/mledu-datasets/sparse-data-embedding/train.tfrecord'
train_path = tf.keras.utils.get_file(train_url.split('/')[-1], train_url)
test_url = 'https://download.mlcc.google.com/mledu-datasets/sparse-data-embedding/test.tfrecord'
test_path = tf.keras.utils.get_file(test_url.split('/')[-1], test_url)

In [5]:
# load data
def parse_fn(record):
    features = {
        "terms": tf.VarLenFeature(dtype=tf.string), # terms are strings of varying lengths
        "labels": tf.FixedLenFeature(shape=[1], dtype=tf.float32) # labels are 0 or 1
    }
  
    parsed_features = tf.parse_single_example(record, features)

    terms = parsed_features['terms'].values
    labels = parsed_features['labels']

    return  {'terms': terms}, labels

def input_fn(input_filenames, num_epochs=None, batch_size=25, shuffle=True):  
    ds = tf.data.TFRecordDataset(input_filenames)
    ds = ds.map(parse_fn)

    if shuffle:
        ds = ds.shuffle(100000)

    ds = ds.padded_batch(batch_size, ds.output_shapes).repeat(num_epochs)

    return ds.make_one_shot_iterator().get_next()

In [6]:
# download the vocabulary file.
terms_url = 'https://download.mlcc.google.com/mledu-datasets/sparse-data-embedding/terms.txt'
terms_path = tf.keras.utils.get_file(terms_url.split('/')[-1], terms_url)

vocab = None
with io.open(terms_path, 'r', encoding='utf8') as f:
    vocab = list(set(f.read().split()))

### Build model with classifier

In [7]:
# build classifier
terms_feature_column = tf.feature_column.categorical_column_with_vocabulary_list(key='terms', 
                                                                                 vocabulary_list=vocab)

terms_embedding_column = tf.feature_column.embedding_column(terms_feature_column, dimension=10)
feature_columns = [terms_embedding_column]

optimizer = tf.train.AdagradOptimizer(learning_rate=0.1)
optimizer = tf.contrib.estimator.clip_gradients_by_norm(optimizer, 5.0)

classifier = tf.estimator.DNNClassifier(
    feature_columns=feature_columns,
    hidden_units=[10, 10],
    dropout=0.3,
    activation_fn=tf.nn.leaky_relu,
    optimizer=optimizer
)

In [8]:
# train
classifier.train(input_fn=lambda: input_fn([train_path], num_epochs=5))

### Evaluate classifier

In [9]:
train_stats = classifier.evaluate(input_fn=lambda: input_fn([train_path], num_epochs=1))
test_stats = classifier.evaluate(input_fn=lambda: input_fn([test_path], num_epochs=1))

train_stats = pd.DataFrame.from_dict(train_stats, orient='index', columns=['train'])
test_stats = pd.DataFrame.from_dict(test_stats, orient='index', columns=['test'])
stats = train_stats.join(test_stats)
stats

,train,test
accuracy,0.956000,0.875840
accuracy_baseline,0.500000,0.500000
auc,0.986418,0.945329
auc_precision_recall,0.986592,0.943740
average_loss,0.139629,0.345184
label/mean,0.500000,0.500000
loss,3.490726,8.629601
precision,0.953172,0.875420
prediction/mean,0.507150,0.502636
recall,0.959120,0.876400


### Export classifier

In [10]:
def serving_input_receiver_fn():
    reviews = tf.placeholder(dtype=tf.string, shape=(None), name='reviews')
    terms = tf.sparse_tensor_to_dense(tf.string_split(reviews), default_value='')
    return tf.estimator.export.ServingInputReceiver({'terms': terms}, {'reviews': reviews})


export_path = classifier.export_savedmodel(export_dir_base='exports',
                                           serving_input_receiver_fn=serving_input_receiver_fn)

export_path = export_path.decode('utf-8')

In [11]:
!!saved_model_cli run --input_exprs 'reviews=["this is a terrible movie", "this is a great movie"]'  \
--dir $export_path --tag_set serve --signature_def predict

['2018-08-28 22:25:24.407476: I tensorflow/core/platform/cpu_feature_guard.cc:141] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA',
 'Result for output key class_ids:',
 '[[0]',
 ' [1]]',
 'Result for output key classes:',
 "[[b'0']",
 " [b'1']]",
 'Result for output key logistic:',
 '[[3.984631e-18]',
 ' [1.000000e+00]]',
 'Result for output key logits:',
 '[[-40.064087]',
 ' [ 24.90329 ]]',
 'Result for output key probabilities:',
 '[[1.0000000e+00 3.9846312e-18]',
 ' [1.5298141e-11 1.0000000e+00]]']